<center><h3> LTL ORDER DETECTION

#### Import Required Python packages

In [1]:
import numpy as np
from faulthandler import is_enabled
from webbrowser import get
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.action_chains import ActionChains
import time
from bs4 import BeautifulSoup
import datetime as dt
from functools import reduce
import warnings
import re
warnings.filterwarnings("ignore")

options = webdriver.ChromeOptions()
options.add_argument(r"--user-data-dir=C:\Users\fahad\AppData\Local\Google\Chrome\User Data\LTL_order_detection")


driver = webdriver.Chrome(options=options)  #Opening selenium
driver.get('https://www.channelspyder.com/ordervending?orderStatus=&warehouse%5B%5D=40&orderDate=&customDate=&otherBuyer=&otherPart=&otherOrder=&limit=200&flagSearch=&errorSearch=&ageSearch=&pendingSearch=')    #Opening Amazon Inventory Page

#### LKQ Shipping

In [2]:
lkq = pd.read_csv("LKQ Shipping.csv")
lkq.head()

,Product,Shipping Method,Ship Cost
0,ABPCV31ELS,SP,16
1,ABPCV31ERS,SP,16
2,ABS149,SP,16
3,AC1000128,LT,225
4,AC1000130,LT,225


#### Create dictionary with Part number as Key and Shipping Method as values

In [3]:
dictionary = lkq.set_index("Product")["Shipping Method"].to_dict()
dictionary

{'ABPCV31ELS': 'SP',
 'ABPCV31ERS': 'SP',
 'ABS149': 'SP',
 'AC1000128': 'LT',
 'AC1000130': 'LT',
 'AC1000133': 'LT',
 'AC1000133C': 'LT',
 'AC1000140': 'LT',
 'AC1000140PP': 'LT',
 'AC1000141': 'LT',
 'AC1000141PP': 'LT',
 'AC1000143': 'LT',
 'AC1000145': 'LT',
 'AC1000149': 'LT',
 'AC1000150': 'LT',
 'AC1000150PP': 'LT',
 'AC1000154': 'LT',
 'AC1000156': 'LT',
 'AC1000156C': 'LT',
 'AC1000157': 'LT',
 'AC1000157PP': 'LT',
 'AC1000158': 'LT',
 'AC1000158C': 'LT',
 'AC1000159': 'LT',
 'AC1000159C': 'LT',
 'AC1000160': 'LT',
 'AC1000162': 'LT',
 'AC1000162PP': 'LT',
 'AC1000163': 'LT',
 'AC1000163PP': 'LT',
 'AC1000168': 'LT',
 'AC1000168C': 'LT',
 'AC1000171R': 'LT',
 'AC1000172': 'LT',
 'AC1000172C': 'LT',
 'AC1000172R': 'LT',
 'AC1000174R': 'LT',
 'AC1000177': 'LT',
 'AC1000177C': 'LT',
 'AC1000178': 'LT',
 'AC1000178C': 'LT',
 'AC1000178OE': 'LT',
 'AC1000179': 'LT',
 'AC1000179PP': 'LT',
 'AC1000180': 'LT',
 'AC1000180C': 'LT',
 'AC1000180OE': 'LT',
 'AC1000180R': 'LT',
 'AC100018

#### Get data from Channel Spyder

In [4]:
def parse_page(driver):
    
    """
    This function parses the webpage data using BeautifulSoup and stores it in a Pandas dataframe.
    
    Parameters:
    driver (WebDriver): A Selenium WebDriver object representing the current webpage.
    
    Returns:
    df (DataFrame): A Pandas DataFrame object containing the data from the webpage.
    """
    # refresh page evverytime before calling function
#     driver.refresh()
    
    # Wait for the page to fully load before trying to parse the data
    WebDriverWait(driver, 300).until( lambda driver: driver.execute_script('return document.readyState') == 'complete')
    
    # Get the HTML source code of the page
    scr = driver.page_source
    
    # Use BeautifulSoup to parse the HTML source code
    soup = BeautifulSoup(scr, 'lxml')
    
    table =soup.find("table", attrs ={"class":"table table-striped table-bordered table-hover"})
    
    # Find all th (table header) elements within the div element
    headers = table.findAll('th')
    
    # Create a list of the header text, stripped of leading/trailing white space
    headerlist =[h.text.strip() for h in headers]
    
    # Find all tr (table row) elements within the div element
    rows = table.findAll('tr')[0:]
    
    # Create a list of lists, where each inner list represents a row of data
    records = [[td.getText().strip() for td in rows[i].findAll('td')[0:]] for i in range(len(rows))]
    
    # Create a Pandas DataFrame from the records list, using the headerlist as the column names
    df = pd.DataFrame(records, columns=headerlist)
    
    return df


#### Data Cleaning

In [5]:
def get_preprocessed_data():
    df = parse_page(driver).loc[1:].drop_duplicates()
    
    ### Remove None rows
    df = df.replace(to_replace=[None], value=np.nan).dropna(axis= 0)
    
    # # Split Order ID into two columns-> Store Name & Order ID.
    df[['Store Name', 'Order ID']] = df["Order ID"].apply(lambda x: pd.Series(str(x).split("\n"))) # Use apply() fnction

    ### Select relevant columns from dataframe
    df = df[["Store Name", "Order ID", "Order Date", "Status", "Qty", "Warehouse", "Total", "Profit $", "Profit %", "Tracking"]]
    
    ### Convert profit feature into numeric
    df['Profit $'] = df["Profit $"].str.replace("$", "", regex = True).astype(float)
    return df

#### Get links of all orders from Channel Spyder Order page

In [6]:
order_links = [element.get_attribute("href") for element in driver.find_elements(By.XPATH, '//*[@id="simpledatatable"]/tbody/tr//td[4]/a')]
order_links

['https://www.channelspyder.com/ordervending/orderView/352711',
 'https://www.channelspyder.com/ordervending/orderView/352700',
 'https://www.channelspyder.com/ordervending/orderView/352670',
 'https://www.channelspyder.com/ordervending/orderView/352637',
 'https://www.channelspyder.com/ordervending/orderView/352617',
 'https://www.channelspyder.com/ordervending/orderView/352614',
 'https://www.channelspyder.com/ordervending/orderView/352605',
 'https://www.channelspyder.com/ordervending/orderView/352601',
 'https://www.channelspyder.com/ordervending/orderView/352597',
 'https://www.channelspyder.com/ordervending/orderView/352571',
 'https://www.channelspyder.com/ordervending/orderView/352566',
 'https://www.channelspyder.com/ordervending/orderView/352550',
 'https://www.channelspyder.com/ordervending/orderView/352533',
 'https://www.channelspyder.com/ordervending/orderView/352515',
 'https://www.channelspyder.com/ordervending/orderView/352481',
 'https://www.channelspyder.com/orderven

In [12]:
df = get_preprocessed_data()
df['Order_links'] = order_links 
display(df.shape)
df

(200, 11)

,Store Name,Order ID,Order Date,Status,Qty,Warehouse,Total,Profit $,Profit %,Tracking,Order_links
1,Part Synergy Amazon,PSA249129812,01-27-2023,New/ Pending,1,,$85.66,0.00,0.00,,https://www.channelspyder.com/ordervending/ord...
3,1-AP Fusion,R112011121,01-27-2023,Sent toWarehouse,1,LKQ,$198.90,0.00,0.00,,https://www.channelspyder.com/ordervending/ord...
5,1-AP Fusion,R078405281,01-27-2023,Sent toWarehouse,1,LKQ,$198.90,0.00,0.00,,https://www.channelspyder.com/ordervending/ord...
7,Part Synergy Amazon,PSA040290660,01-27-2023,Sent toWarehouse,1,LKQ,$38.48,0.00,0.00,,https://www.channelspyder.com/ordervending/ord...
9,1-AP Fusion,R644423729,01-27-2023,Sent toWarehouse,1,LKQ,$190.97,0.00,0.00,,https://www.channelspyder.com/ordervending/ord...
...,...,...,...,...,...,...,...,...,...,...,...
391,Part Synergy Amazon,PSA663236242,01-18-2023,Shipped,1,LKQ,$80.77,8.19,10.14,1ZR56W920301505864,https://www.channelspyder.com/ordervending/ord...
393,Part Synergy Amazon,PSA248571428,01-18-2023,Shipped,1,LKQ,$80.77,7.77,9.62,1Z2197180306252817,https://www.channelspyder.com/ordervending/ord...
395,Part Synergy Amazon,PSA412376269,01-18-2023,Shipped,1,LKQ,$65.14,7.41,11.38,1Z3058250303728352,https://www.channelspyder.com/ordervending/ord...
397,Baabs Auto,PB230959386655,01-18-2023,Shipped,1,LKQ,$79.54,12.48,15.69,1Z1601XX0305182247,https://www.channelspyder.com/ordervending/ord...


#### Collect data for each order from order page on Channel Spyder and save in dataframe

In [8]:
## Store the path to inventory page in variable->Inventory_page
Inventory_page = driver.current_window_handle

In [9]:
ordered_date = []; number_of_items = []; order_page = []; shipping_address = []; store_name = []; channel_spyder = []
shipping_cost = []; shipping_carrier = []; order_id = [];shipping_method = []; channel_order_no = []


# Find the index of the last checked order
order_checked = df[df["Order ID"] == checked].index
# Check if the order_checked exists in dataframe
if len(order_checked) > 0:
    
    # Get the index of the last row to drop
    last_index_to_drop = order_checked[0]
    # Drop all rows below the order_checked
    df = df.drop(df.index[last_index_to_drop+1:])
    
for index, link in enumerate(list(df.Order_links.values)):
    
    ## switches the browser to a new window
    driver.switch_to.new_window()  
    # Navigates the new window browser to the URL specified in the "href" variable.
    driver.get(link)
    print('Opening Page: ', index)
    time.sleep(1)

    part_number = driver.find_elements(By.XPATH, '//*[@id="simpledatatable"]/tbody/tr/td[2]/span/a') #Getting all download buttons
    ware_house = driver.find_elements(By.XPATH, '//*[@id="simpledatatable"]/tbody/tr/td[2]/span[2]')
    p_list = ["".join(element.get_attribute("innerText").strip()) for element in part_number]
    w_list = ["".join(element.get_attribute("innerText").strip()) for element in ware_house]
    
    print(p_list)
    print(w_list)
    p_list_update = [p for p, w in dict(zip(p_list, w_list)).items() if w == "455"]
    print(p_list_update)
    
    
    ## Check for all the information of LKQ order using it's partnumber 
    try:
        for category in p_list_update:
            Shipping = dictionary[category]
            print(Shipping)



            if (Shipping == "LT"):
                OI = driver.find_element(By.XPATH, '//*[@id="MainPageContent"]/div/div[3]/div[3]/div[1]')
                order_info = OI.get_attribute('innerText') 

                CI = driver.find_element(By.XPATH, '//*[@id="MainPageContent"]/div/div[3]/div[3]/div[2]')
                customer_info = CI.get_attribute('innerText') 

                SI = driver.find_element(By.XPATH, '//*[@id="MainPageContent"]/div/div[3]/div[3]/div[3]')
                shipping_info = SI.get_attribute('innerText') 

                ship_status = driver.find_element(By.XPATH, '//*[@id="simpledatatable"]/tbody/tr/td[1]/b').get_attribute("innerText")

                store = driver.find_element(By.XPATH, '//*[@id="MainPageContent"]/div/div[3]/div[2]/div/div/div/div/div[1]/span[1]').get_attribute("innerText").split(':')[1].strip()
                order_no = driver.find_element(By.XPATH, '//*[@id="MainPageContent"]/div/div[3]/div[3]/div[1]/div/div[2]/div/div/div/p[1]/b/span').get_attribute("innerText")
                
                try:
                    order_link = driver.find_element(By.XPATH, '//*[@id="MainPageContent"]/div/div[3]/div[2]/div/div/div/div/div[1]/span[2]/a[1]').get_attribute("href")
                except:
                    order_link = ""
                    
                print(order_no)
                print(order_link)

                # Extract the ordered date from the order_info string using regular expressions
                # and store it in the "date" variable; (same process in other info)
                date = re.findall(r'Ordered Date : (\d{2}-\d{2}-\d{4} \d{2}:\d{2}:\d{2})', order_info)[0]
                oid = "".join(re.search(r'Order ID : (.*?) ', order_info).group(1).strip())
                num_items = "".join(re.search(r'Number of Items : (.*?)\n', order_info).group(1).strip())
                address = "".join(re.search(r'Ship Address 1 : (.*?)\n', shipping_info).group(1).strip())


                try:
                    cost = re.search(r'\$\d+\.\d+', ship_status).group()
                    carrier = re.search(r'Carrier : (.*?) ', ship_status).group(1)
                except:
                    carrier = ""
                    cost = ""


                # Append the extracted values to corresponding lists
                order_id.append(oid)
                ordered_date.append(date)
                number_of_items.append(num_items)
                shipping_address.append(address)
                shipping_method.append(Shipping)
                shipping_carrier.append(carrier)
                shipping_cost.append(cost)
                order_page.append(order_link)
                store_name.append(store)
                channel_order_no.append(order_no)
                channel_spyder.append(link)
    except:
        print("Part Number not found in LTL shipping orders file")

    # Close the new open window of the browser
    driver.close()  
        
    # switches the browser back to the original window-> Inventory page 
    driver.switch_to.window(Inventory_page)
    

Opening Page:  0
['GM1039176']
['455']
['GM1039176']
SP
Opening Page:  1
['ALY69892U20']
['455']
['ALY69892U20']
SP


In [13]:
### Create DataFrame for output
LTL_orders = pd.DataFrame({"Order ID": order_id,
                    "Order Date": ordered_date,
                  "No. of Items": number_of_items,
                  "Shipping Address": shipping_address,
                  "Shipping Cost": shipping_cost,
                  "Shipping Carrier": shipping_carrier,
                  "Shipping Method": shipping_method,
                  "Channel Spyder":channel_spyder,
                  "Selling Store":store_name,
                  "Marketplace order no":channel_order_no,
                  "Marketplace order path":order_page,

                          })


display(LTL_orders)
checked = df["Order ID"].iloc[0]

,Order ID,Order Date,No. of Items,Shipping Address,Shipping Cost,Shipping Carrier,Shipping Method,Channel Spyder,Selling Store,Marketplace order no,Marketplace order path


<center><h2> THE END